## Importuri necesare 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Citirea datelor

In [ ]:
import pandas as pd
data_path = ''
train_data_df = pd.read_csv(os.path.join(data_path, 'train_data.csv'))

In [ ]:
train_data_df

In [ ]:
print('Distributia etichetelor in datele de antrenare \n', train_data_df['label'].value_counts())

In [ ]:
# fiecare limba este reprezentata in mod egal
train_data_df['language'].value_counts()

## Codificarea etichetelor din string in int

In [ ]:
#Exercitiu: codificam etichetele / labels in valori cu numere intregi dela 0 la N
# codificam etichetele / labels in valori cu numere intregi dela 0 la N
# ar putea fi exercitiu la clasa ca sa se obisnuiasca cu dictionare
etichete_unice = train_data_df['label'].unique()
label2id = {}
id2label = {}
for idx, eticheta in enumerate(etichete_unice):
    #TODO:
    pass

print(label2id)
print(id2label)

In [ ]:
#Exercitiu: aplicati dictionarul label2id peste toate etichetele din train
labels = []
#TODO:

print(labels[:10])

## Pre-procesarea datelor

- extragem informațiile necesare din text
- eliminăm semnele de punctuație
- facem tokenizare (împărțire în cuvinte)
- vezi tokenizatori in [NLTK](https://www.nltk.org/api/nltk.tokenize.html)
- vezi tokenizatori in [spaCy](https://spacy.io/)
- vezi tokenizatori precum [BPE, WordPiece, SentencePiece](https://huggingface.co/docs/transformers/tokenizer_summary)

In [ ]:
import re

def proceseaza(text):
    """Functie simpla de procesare a textului.
    Sugestii:
    - cum procesati \n new lines? (vezi functia strip())
    - cum procesati empty token ''
    - puteti introduce un tokenizator din nltk
        https://www.nltk.org/api/nltk.tokenize.html
    - puteti elimina sau pastra doar stop-words
    """
    text = re.sub("[-.,;:!?\"\'\/()_*=`]", "", text)
    text_in_cuvinte = text.split(' ')
    return text_in_cuvinte

# cuvintele rezultate din functia de preprocesare:
exemple_italian = train_data_df[train_data_df['language'] == 'italiano']
print(exemple_italian)


In [ ]:
text_italian = exemple_italian['text'].iloc[0]
print(proceseaza(text_italian)[:13])

### Aplicam functia de preprocesarea intregului set de date

In [ ]:
# Exercitiu: aplicati functia pe fiecare exemplu de antrenare
data = []
# TODO
print(data[0])

## Împărțirea datelor în train, validare și test

O împărțire brutală poate fi cea în funcție de ordinea în care apar datele.
- cum poate afecat acest tip de împărțire rezultatele dacă datele sunt sortate după etichetă?

In [ ]:
print(len(data))

# putem imparti datele de antrenare astfel:
# 20% date de test din total 
# 15% date de validare din ce ramane dupa ce scoatem datele de test 

nr_test = int(20/100 * len(train_data_df))
print("Nr de date de test: ", nr_test)

nr_ramase = len(data) - nr_test
nr_valid = int(15/100 * nr_ramase)
print("Nr de date de validare: ", nr_valid)

nr_train = nr_ramase - nr_valid
print("Nr de date de antrenare: ", nr_train)



In [ ]:
# facem impartirea in ordinea in care apar datele

train_data = data[:nr_train]
train_labels = labels[:nr_train]

valid_data = data[nr_train : nr_train + nr_valid]
valid_labels = labels[nr_train : nr_train + nr_valid]

test_data = data[nr_train + nr_valid: ]
test_labels = labels[nr_train + nr_valid:]


print(f'Nr de exemple de train {len(train_labels)}')
print(f'Nr de exemple de validare {len(valid_labels)}')
print(f'Nr de exemple de test {len(test_labels)}')

In [ ]:
#Exercitiu: vedeti cum sunt distribuite etichetele in train, valid, test (incercati bincount)

## Bag of Words

În cadrul acestei secțiuni vom face numărarea aparițiilor tuturor cuvintelor din datele noastre. Pentru o evaluare justă nu ar fi indicat să includem si cuvintele din datele de test.

In [ ]:
from collections import Counter

# vom folosi o structura de date nativa din python
# care functioneaza ca un dictionar care numara elementele hashable
# dintr-o colectie
ctr = Counter(['eggs', 'ham', 'eggs', 'egg'])
print(ctr['bacon'])
print(ctr['eggs'])

ctr = Counter([(0,1), (0,0), (0,1)])
print(ctr[(0,0)])
print(ctr[(2,2)])


In [ ]:
ctr = Counter(data[9234])
print(ctr.keys())
print(ctr.values())
# care este cel mai frecvent cuvant in data[5]?

### Frecventa cuvintelor din setul de antrenare

In [ ]:
counter = Counter()
#TODO:

print(counter.most_common(10))


### Reprezentarea datelor sub forma vectoriala

- sa presupunem ca folosim primele N cuvinte non-nule ca caracteristici pentru fiecare text


In [ ]:
N = 10
cuvinte_caracteristice = []
for cuvant, frecventa in counter.most_common(N):
    if cuvant.strip():
        cuvinte_caracteristice.append(cuvant)
print(cuvinte_caracteristice)

- fiecarui cuvant îi atribuim un id în funcție de poziția pe care se află
- ordinea in care sunt stocate cheile intr-un dictionar este arbitrara 
- iar o lista este un obiect mutabil in care ordinea elementelor se poate schimba oricand
- cel mai sigur este sa construim o mapare intre cuvinte si un id care sa reprezinte pozitia in vectorul de caracteristici


In [ ]:
word2id = {}
id2word = {}
for idx, cuv in enumerate(cuvinte_caracteristice):
    word2id[cuv] = idx
    id2word[idx] = cuv

print(word2id)
print(id2word)

- cand trebuie sa reprezentam un text sub forma vectoriala, ne raportam doar la cuvintele cheie pe care le folosim ca caracteristici
- id-ul reprezinta pozitia in vector unde vom stoca aparitiile fiecarui cuvant

In [ ]:
# 1. numaram toate cuvintele din text
ctr = Counter(train_data[1])

# 2. prealocam un array care va reprezenta caracteristicel noastre
features = np.zeros(len(cuvinte_caracteristice))

# 3. umplem array-ul cu valorile obtinute din counter
# fiecare pozitie din array trebuie sa reprezinte frecventa
# aceluiasi cuvant in toate textele
for idx in range(0, len(features)):
    # obtinem cuvantul pentru pozitia idx
    cuvant = id2word[idx]
    # asignam valoarea corespunzatoare frecventei cuvantului
    features[idx] = ctr[cuvant]

print(features)
print([id2word[idx] for idx in range(0, len(features))])

### Punem totul cap la cap sub forma de functii

In [ ]:
def count_most_common(how_many, texte_preprocesate):
    """Functie care returneaza cele mai frecvente cuvinte.
    """
    #TODO
    return cuvinte_caracteristice

In [ ]:
def build_id_word_dicts(cuvinte_caracteristice):
    '''Dictionarele word2id si id2word garanteaza o ordine
    pentru cuvintele caracteristice.
    '''
    #TODO
    return word2id, id2word

In [ ]:
def featurize(text_preprocesat, id2word):
    """Pentru un text preprocesat dat si un dictionar
    care mapeaza pentru fiecare pozitie ce cuvant corespunde,
    returneaza un vector care reprezinta
    frecventele fiecarui cuvant.
    """
    ctr = Counter(text_preprocesat)
    features = np.zeros(len(id2word))
    #TODO
    return features

In [ ]:
def featurize_multi(texte, id2word):
    '''Pentru un set de texte preprocesate si un dictionar
    care mapeaza pentru fiecare pozitie ce cuvant corespunde,
    returneaza matricea trasaturilor tuturor textelor.
    '''
    all_features = []
    for text in texte:
        all_features.append(featurize(text, id2word))
    return np.array(all_features)

In [ ]:
cuvinte_caracteristice = count_most_common(30, train_data)
print(len(cuvinte_caracteristice))
word2id, id2word = build_id_word_dicts(cuvinte_caracteristice)

X_train = featurize_multi(train_data, id2word)
X_valid = featurize_multi(valid_data, id2word)
X_test = featurize_multi(test_data, id2word)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import svm

model = svm.LinearSVC(C=1)

model.fit(X_train, train_labels)
vpreds = model.predict(X_valid)
tpreds = model.predict(X_test)

print(accuracy_score(valid_labels, vpreds))
print(accuracy_score(test_labels, tpreds))

## Exemplu de lucru cu stop words in nltk

In [ ]:
# luat in calcul cuvintele functionale, pot creste sau scadea acuratetea
# in cazul de fata, cuvintele functionale (stop words) indica elemente de gramatica
# care sunt specifice textelor de la Scotieni, Englezi, Irlandezi
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
print(stopwords.words('french'))